# SQLite 和 MySQL 的 python 操作

SQLite 是一种嵌入式数据库，它的数据库是一个文件。使用 Python 操作 SQLite 数据库的基本流程如下：

- 通过 `sqlite3.open()` 创建于数据路的连接对象 connectoin;
- 通过 `connection.cursor()` 创建光标对象 cursor;
- 通过 `cursor.execute()` 执行 SQL 语句；
- 通过 `connection.commit()` 提交当前的事务，或者通过 `cursor.fetchall()` 获得查询结果；
- 通过 `connection.close()` 关闭于数据库文件的连接。

In [2]:
import sqlite3

conn = sqlite3.connect("test.db")
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS students (sid INTEGER PRIMARY KEY, name TEXT)")
conn.commit()

conn.close()

这里在当前工作目录下创建了一个数据库文件 `test.db`，另外还可以使用`":memory:"` 建立内存数据库。

> 内存数据库是一种将全部内容存放在内存中，而非传统数据库那样存放在外部存储器中的数据库。这种数据库的读写性能很高，主要用于在对性能要求极高的环境中，但是在服务器关闭后会立刻丢失全部存储的数据。

In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE students (sid INTEGER PRIMARY KEY, name TEXT)")
conn.commit()

c.execute("INSERT INTO students VALUES(?, ?)", (1, "Alice"))
c.execute("INSERT INTO students VALUES(?, ?)", (2, "Bob"))
c.execute("INSERT INTO students VALUES(?, ?)", (3, "Peter"))

c.execute("DELETE FROM students WHERE sid = ?", (1,))
c.execute("UPDATE students SET name = ? WHERE sid = ?", ("Mark", 3))

conn.commit()

c.execute("SELECT * FROM students")
print(c.fetchall())

conn.close()

[(2, 'Bob'), (3, 'Mark')]


`fetchall` 返回满足所有要求的 records 列表。

#### 封装成 Python 函数

In [5]:
import sqlite3
import os, sys

def initialize(conn):
    c = conn.cursor()
    c.execute("CREATE TABLE students (sid INTEGER PRIMARY KEY, name TEXT)")
    conn.commit()

def insert(conn, sid, name):
    c = conn.cursor()
    t = (sid, name)
    c.execute("INSERT INTO students VALUES (?, ?)", t)
    conn.commit()
    
def delete(conn, sid):
    c = conn.cursor()
    t = (sid, )
    c.execute("DELETE FROM students WHERE sid = ?", t)
    conn.commit()
    
def update(conn, sid, name):
    c = conn.cursor()
    t = (name, sid)
    c.execute("UPDATE students SET name = ? WHERE sid = ?", t)
    conn.commit()
    
def display(conn):
    c = conn.cursor()
    c.execute("SELECT * FROM students");
    print(c.fetchall())
    
db_name = ":memory:"
conn = sqlite3.connect(db_name)

initialize(conn)

print("Insert 3 records.")
insert(conn, 1, "Alice")
insert(conn, 2, "Bob")
insert(conn, 3, "Peter")
display(conn)

print("Delete the record where sid = 1.")
delete(conn, 1)
display(conn)

print("Update the record where sid = 3.")
update(conn, 3, "Mark")
display(conn)

conn.close()

Insert 3 records.
[(1, 'Alice'), (2, 'Bob'), (3, 'Peter')]
Delete the record where sid = 1.
[(2, 'Bob'), (3, 'Peter')]
Update the record where sid = 3.
[(2, 'Bob'), (3, 'Mark')]


## MySQL

MySQL 和 SQLite 都是关系型数据库，他们的操作方法也是大同小异的。首先建立连接对象与光标对象，用`execute()` 执行 SQL 语句，`commit()` 提交任务，`fetchall()` 获得查询结果。 

MySQL 以及 mysql-connector-python 的安装
在 Mac 上可以通过 brew 安装，或者下载 dmg 安装文件，还可以通过 XAMMPP 来进行安装。
mysql-connector-python 通过 pip 安装出现了点问题。我下载了 source code 进行安装，很方便。
上面这两个在 anaconda 上都可以进行安装。

### MySQL server 启动
我们需要先在本地运行起来 mysql server 然后才能够连接 server 进行一些操作。

```
mysqld start
mysql.server start       #1. 启动
mysql.server stop        #2. 停止
mysql.server restart     #3. 重启
```

启动完成之后，还需要进行一些初始设置:

```
mysql_secure_installation

Enter current password for root (enter for none):
# 解释：输入当前 root 用户密码，默认为空，直接回车。

Set root password? [Y/n]  y
# 解释：要设置 root 密码吗？输入 y 表示愿意。

Remove anonymous users? [Y/n]  y
# 解释：要移除掉匿名用户吗？输入 y 表示愿意。

Disallow root login remotely? [Y/n]  y
# 解释：不想让 root 远程登陆吗？输入 y 表示愿意。

Remove test database and access to it? [Y/n]  y
# 解释：要去掉 test 数据库吗？输入 y 表示愿意。

Reload privilege tables now? [Y/n]  y
# 解释：想要重新加载权限吗？输入 y 表示愿意。
```

设置了用户名密码之后我们就可以登陆 mysql server, `mysql -u root -p logitech`, 然后你就可创建第一个 database employees, `CREATE DATABASE employees`; 然后进入数据库 `USE employees`。

接着你就可以创建 tables 等等。不过这部分我们将在 python 中完成。

最后你想要推出 mysql 环境使用命令 `exit;`

### Python 脚本连接数据库
首先运行 `mysql.server start`,  然后使用如下 Python 脚本连接 mysql server。

In [1]:
import mysql.connector

# Connect with the MySQL Server
config = {
    'user': 'root',
    'password': 'logitech',
    'host': 'localhost',
    'database': 'employees',
    'raise_on_warnings': True,
}

cnx = mysql.connector.connect(**config)

cnx.close()

上面的代码直接连接到一个 database `employees`，另外你也可以先连接 MySQL server, 然后选择某一个 database，如果这个 database 不存在，就按名字创建。

In [5]:
import mysql.connector
from mysql.connector import errorcode

DB_NAME = 'employees'

cnx = mysql.connector.connect(user='root', password='logitech')
cursor = cnx.cursor()

def create_database(cursor):
    try:
        cursor.execute(
        'CREATE DATABASE {} DEFAULT CHARACTER SET "utf8"'.format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cnx.database = DB_NAME
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

# cursor.close()
# cnx.close()

上面的代码连接到了 employees 数据库，接下来我们就可以创建具体的数据表格了。

In [7]:
TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE `employees` ("
    "  `emp_no` int(11) NOT NULL AUTO_INCREMENT,"
    "  `birth_date` date NOT NULL,"
    "  `first_name` varchar(14) NOT NULL,"
    "  `last_name` varchar(16) NOT NULL,"
    "  `gender` enum('M','F') NOT NULL,"
    "  `hire_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE `departments` ("
    "  `dept_no` char(4) NOT NULL,"
    "  `dept_name` varchar(40) NOT NULL,"
    "  PRIMARY KEY (`dept_no`), UNIQUE KEY `dept_name` (`dept_name`)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE `salaries` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `salary` int(11) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`from_date`), KEY `emp_no` (`emp_no`),"
    "  CONSTRAINT `salaries_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE `dept_emp` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `dept_no` char(4) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`dept_no`), KEY `emp_no` (`emp_no`),"
    "  KEY `dept_no` (`dept_no`),"
    "  CONSTRAINT `dept_emp_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE,"
    "  CONSTRAINT `dept_emp_ibfk_2` FOREIGN KEY (`dept_no`) "
    "     REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE `dept_manager` ("
    "  `dept_no` char(4) NOT NULL,"
    "  `emp_no` int(11) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`dept_no`),"
    "  KEY `emp_no` (`emp_no`),"
    "  KEY `dept_no` (`dept_no`),"
    "  CONSTRAINT `dept_manager_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE,"
    "  CONSTRAINT `dept_manager_ibfk_2` FOREIGN KEY (`dept_no`) "
    "     REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE `titles` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `title` varchar(50) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date DEFAULT NULL,"
    "  PRIMARY KEY (`emp_no`,`title`,`from_date`), KEY `emp_no` (`emp_no`),"
    "  CONSTRAINT `titles_ibfk_1` FOREIGN KEY (`emp_no`)"
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

for name, ddl in TABLES.items():
    try:
        print("Creating table {}: ".format(name), end="")
        cursor.execute(ddl)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: OK
Creating table departments: OK
Creating table salaries: OK
Creating table dept_emp: OK
Creating table dept_manager: OK
Creating table titles: OK


这里做为练习，我先直接向每个表格中倒入一些数据，然后在进行操作。